In [1]:
import numpy as np
import tensorflow as tf

In [2]:
train_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\32-bit\\train_32bit.txt',delimiter=",")
test_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\32-bit\\test_32bit.txt',delimiter=",")
validation_vector=np.loadtxt('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro4\\PRNG-Dataset\\32-bit\\val_32bit.txt',delimiter=",")

In [3]:
train_data=train_vector[:,:-1]
#print(train_data[:2])
train_label=train_vector[:,-1]
#print(train_label)
test_data=test_vector[:,:-1]
#print(test_data[:2])
test_label=test_vector[:,-1]
#print(test_label)
val_data=validation_vector[:,:-1]
val_label=validation_vector[:,-1]

In [4]:
v1=4
v2=8

In [6]:
x_placeholder=tf.placeholder(tf.float32,[None,v1,v2,1])
y_placeholder=tf.placeholder(tf.int32,[None])
conv0=tf.layers.conv2d(x_placeholder,20,3,activation=tf.nn.relu,padding='same')
pool0=tf.layers.max_pooling2d(conv0,[2,2],[2,2])#4,8
conv1=tf.layers.conv2d(pool0,60,2,activation=tf.nn.relu,padding='same')
pool1=tf.layers.max_pooling2d(conv1,[2,2],[2,2])#2,4
#conv2=tf.layers.conv2d(pool0,60,2,activation=tf.nn.relu,padding='same')
#pool2=tf.layers.max_pooling2d(conv2,[2,2],[2,2])

flatten=tf.layers.flatten(pool1)
print(flatten)
#fc=tf.layers.dense(flatten,20,activation=tf.nn.relu)
logits=tf.layers.dense(flatten,8)
predicted_labels=tf.arg_max(logits,1)
losses=tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y_placeholder,8),logits=logits)
mean_loss=tf.reduce_mean(losses)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(losses)

Tensor("flatten_1/Reshape:0", shape=(?, 120), dtype=float32)


In [19]:
val_x=np.reshape(val_data,[-1,v1,v2,1])
val_y=val_label-1
train_x=np.reshape(train_data,[-1,v1,v2,1])
train_y=train_label-1
test_x=np.reshape(test_data,[-1,v1,v2,1])
test_y=test_label-1
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    val_feed_dict={x_placeholder:val_x,y_placeholder:val_y}
    train_feed_dict={x_placeholder:train_x,y_placeholder:train_y}
    max_acc=0
    for step in range(200):
        _,mean_loss_val,predicted_labels_val=sess.run([optimizer,mean_loss,predicted_labels],feed_dict=train_feed_dict)
        predicted_labels_validation=sess.run([predicted_labels],feed_dict=val_feed_dict)
        acc=np.sum(predicted_labels_validation==val_y)/val_y.shape[0]
        print(step,mean_loss_val,acc)
        if max_acc<acc:
            max_acc=acc
            print("saving model")
            saver.save(sess,'model/')

0 2.090288 0.1417
saving model
1 2.0778728 0.1812
saving model
2 2.0686455 0.1856
saving model
3 2.0578282 0.1789
4 2.0442903 0.1775
5 2.0280387 0.1793
6 2.0097532 0.1794
7 1.9920534 0.1805
8 1.9791026 0.1801
9 1.9756372 0.1813
10 1.9801017 0.1839
11 1.9833453 0.1753
12 1.9790905 0.1775
13 1.9743369 0.1823
14 1.9683472 0.1826
15 1.9624364 0.1821
16 1.9602405 0.184
17 1.9592398 0.1846
18 1.9594753 0.1861
saving model
19 1.9597887 0.1858
20 1.9587641 0.186
21 1.956621 0.1864
saving model
22 1.9530841 0.1896
saving model
23 1.9504488 0.1887
24 1.9484904 0.1881
25 1.9477792 0.1848
26 1.9473664 0.1885
27 1.9469213 0.1881
28 1.9461313 0.1878
29 1.944377 0.1879
30 1.9428519 0.1883
31 1.9417636 0.1885
32 1.9412144 0.1886
33 1.9411018 0.1887
34 1.9408019 0.1864
35 1.9401062 0.187
36 1.9390258 0.185
37 1.9383268 0.188
38 1.938033 0.1863
39 1.937963 0.1881
40 1.9377083 0.1849
41 1.9371995 0.1843
42 1.9366683 0.1867
43 1.9361931 0.1866
44 1.9360508 0.189
45 1.9358773 0.1886
46 1.9356107 0.1892
47 

In [20]:
with tf.Session() as sess:
    saver.restore(sess,'model/')
    test_feed_dict={x_placeholder:test_x,y_placeholder:test_y}
    predicted_labels_val=sess.run(predicted_labels,feed_dict=test_feed_dict)
    acc=np.sum(predicted_labels_val==test_y)/test_x.shape[0]
    print(acc)

INFO:tensorflow:Restoring parameters from model/
0.189
